In [2]:
import torch
import tensorflow as tf

device=torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(device)
# Optional: Force GPU/CPU usage
if tf.config.list_physical_devices('GPU'):
    device = '/GPU:0'
    print("Using GPU")
else:
    device = '/CPU:0'
    print("Using CPU")

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from collections import Counter

# Load data
X_train = np.load('/kaggle/input/classifier-data/X_train.npy')
y_train_raw = np.load('/kaggle/input/classifier-data/y_train.npy')
X_test = np.load('/kaggle/input/classifier-data/X_test.npy')
y_test_raw = np.load('/kaggle/input/classifier-data/y_test.npy')

# Convert to binary labels
y_train = (y_train_raw.sum(axis=1) > 0).astype(int)
y_test = (y_test_raw.sum(axis=1) > 0).astype(int)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Class weights to handle imbalance
counter = Counter(y_train)
weight_for_0 = (1 / counter[0]) * (len(y_train) / 2.0)
weight_for_1 = (1 / counter[1]) * (len(y_train) / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}

# Model definition
model = Sequential([
    Dense(64, activation='relu', kernel_regularizer=l2(0.001), input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

# Train model
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=60,
    batch_size=16,
    class_weight=class_weight,
    #callbacks=[early_stop, lr_schedule],
    verbose=1
)

# Predict & evaluate
y_pred_probs = model.predict(X_test_scaled)
y_pred = (y_pred_probs > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


cuda
Using GPU
Epoch 1/60


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 263ms/step - accuracy: 0.4660 - loss: 1.4057 - val_accuracy: 0.4231 - val_loss: 1.0440
Epoch 2/60
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5107 - loss: 1.0109 - val_accuracy: 0.4231 - val_loss: 1.0399
Epoch 3/60
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5302 - loss: 1.3281 - val_accuracy: 0.4231 - val_loss: 1.0364
Epoch 4/60
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5773 - loss: 1.0853 - val_accuracy: 0.4615 - val_loss: 1.0333
Epoch 5/60
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4755 - loss: 1.2767 - val_accuracy: 0.4615 - val_loss: 1.0289
Epoch 6/60
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4558 - loss: 1.1454 - val_accuracy: 0.4615 - val_loss: 1.0252
Epoch 7/60
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3978 - loss: 1.2217 - val_accuracy: 0.4615 - val_loss: 1.0181
Epoch 8/60
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5525 - loss: 1.1365 - val_accuracy: 0.4615 - val_loss: 1.0126
Epoch 9/60
7/7 ━━

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Dropout, Flatten,
                                     Dense, BatchNormalization)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# Load data
X_train = np.load('/kaggle/input/classifier-data/X_train.npy')
X_test = np.load('/kaggle/input/classifier-data/X_test.npy')
y_train = np.load('/kaggle/input/classifier-data/y_train.npy')  # Assuming labels are in a separate file
y_test = np.load('/kaggle/input/classifier-data/y_test.npy')  # Assuming labels are in a separate file

# Preprocessing: Normalize the image data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Check the shape of the data
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

# Model architecture
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.25),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.25),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilation of the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks for early stopping and learning rate adjustment
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=3, factor=0.5)
]

# Training the model
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2,  # Using 20% of data for validation
    callbacks=callbacks
)

# Model Evaluation
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Evaluate performance
print(f"F1 Score: {f1_score(y_test, y_pred)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")


Train shape: (126, 360), Test shape: (33, 360)
Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 360), dtype=float32). Expected shape (None, 128, 128, 3), but input has incompatible shape (None, 360)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 360), dtype=float32)
  • training=True
  • mask=None